In [2]:
using Random, Distributions, Plots

ns = [50, 100, 500, 2000, 10000]
num_datasets = 50
num_runs = 10

function round_array(xs::Vector{Float64}, R::Int)::Vector{Int}
    new_xs = Vector{Int}([])
    for x in xs
        if x < 1
            push!(new_xs, 1)
        elseif x > R
            push!(new_xs, Int(round(R)))
        else
            push!(new_xs, Int(round(x)))
        end
    end
    new_xs
end

function sign(z::Int)::Int
    if z > 0
        return 1
    elseif z < 0
        return -1
    else
        return 0
    end
end

function score(y::Int, xs::Vector{Int})::Int
    sum = 0
    for i in xs
        sum += sign(y - i)
    end
    -abs(sum)
end

function scores(xs::Vector{Int64})
    """
    """
    qs = Vector{Float64}([])
    for x in xs
        push!(qs, score(x, xs))
    end
    qs
end

scores (generic function with 1 method)

In [26]:
function RNM(qs::Vector{Float64}, δ::Int64=1, ϵ::Float64=0.1)
    """
    """
    zs = rand(Exponential(2*δ/ϵ), length(qs))
    output = qs + zs
    return argmax(output)
end

function plot_avgs(ns, err_arr, ylabel, title) 
    plot(string.(ns), err_arr[1], lc=:blue, primary=false, ylims=(0, 25))
    plot!(string.(ns), err_arr[1], seriestype=:scatter, mc=:blue, label="R=2^7")
    
    plot!(string.(ns), err_arr[2], lc=:red, primary=false)
    plot!(string.(ns), err_arr[2], seriestype=:scatter, mc=:red, label="R=2^10")
    
    plot!(string.(ns), err_arr[3], lc=:green, primary=false)
    plot!(string.(ns), err_arr[3], seriestype=:scatter, mc=:green, label="R=2^16", legend=:bottomleft)
    xlabel!("n")
    ylabel!(ylabel)
    title!(title)
    savefig(title * ".png")
end

function plot_avgs_bimodal(ns, err_arr, ylabel, title) 
    plot(string.(ns), err_arr[1], lc=:blue, primary=false, ylims=(0, 25))
    plot!(string.(ns), err_arr[1], seriestype=:scatter, mc=:blue, label="k=10")
    
    plot!(string.(ns), err_arr[2], lc=:red, primary=false)
    plot!(string.(ns), err_arr[2], seriestype=:scatter, mc=:red, label="k=100")
    
    plot!(string.(ns), err_arr[3], lc=:green, primary=false)
    plot!(string.(ns), err_arr[3], seriestype=:scatter, mc=:green, label="k=200", legend=:bottomleft)
    xlabel!("n")
    ylabel!(ylabel)
    title!(title)
    savefig(title * ".png")
end

function gaussian_exp(ϵ, ns, num_datasets, num_runs)
    """
    """
    Rs = [2^7, 2^10, 2^16]
    all_err_means = []
    all_err_stds = []
    all_within_run_stds = []
    for R in Rs
        error_means = []
        error_stds = []
        R_within_run_stds = []
        for n in ns
            errors = Vector{Int}([])
            within_run_stds = []
            for i=1:num_datasets
                dataset = rand(Normal(R/4, R/sqrt(10)), n)
                xs = round_array(dataset, R)
                qs = scores(xs)
                true_median = xs[argmax(qs)]
                
                within_run_errors = []
                for run=1:num_runs
                    run_median = xs[RNM(qs)]
                    error = abs(score(true_median, xs) - score(run_median, xs))
                    push!(errors, error)
                    push!(within_run_errors, error)
                end
                push!(within_run_stds, std(within_run_errors))
            end 
        
            push!(error_means, mean(errors))
            push!(error_stds, std(errors))
            push!(R_within_run_stds, mean(within_run_stds))
        end
        push!(all_within_run_stds, R_within_run_stds)
        push!(all_err_means, error_means)
        push!(all_err_stds, error_stds)
    end    
    plot_avgs(ns, all_err_means, "Avg score error",
        "Avg score error for Gaussian runs")
    plot_avgs(ns, all_err_stds, "Std Dev of score error",
        "Std Dev of score error for Gaussian runs")
    plot_avgs(ns, all_within_run_stds, "Avg within-run score error",
        "Avg within-run score error for Gaussian runs")
end

function poisson_exp(ϵ, ns, num_datasets, num_runs)
    Rs = [2^7, 2^10, 2^16]
    all_err_means = []
    all_err_stds = []
    for R in Rs
        error_means = []
        error_stds = []
        for n in ns
            errors = Vector{Int}([])
            for i=1:num_datasets
                dataset = Vector{Float64}(rand(Poisson(50), n))
                xs = round_array(dataset, R)
                qs = scores(xs)
                true_median = xs[argmax(qs)]
                
                for run=1:num_runs
                    run_median = xs[RNM(qs)]
                    error = abs(score(true_median, xs) - score(run_median, xs))
                    push!(errors, error)
                end
            end  
            push!(error_means, mean(errors))
            push!(error_stds, std(errors))
        end
        push!(all_err_means, error_means)
        push!(all_err_stds, error_stds)
    end    
    plot_avgs(ns, all_err_means, "Avg score error",
        "Avg score error for Poisson runs")
    plot_avgs(ns, all_err_stds, "Std Dev of score error",
        "Std Dev of score error for Poisson runs")
end

function draw_uniform_bimodal(left, right, n)
    """
    """
    dataset = Vector{Float64}()
    for i=1:n
        if rand() > 0.5
            push!(dataset, left)
        else
            push!(dataset, right)
        end
    end
    dataset
end

function bimodal_exp(ϵ, ns, num_datasets, num_runs)
    R = 2^10
    ks = [10, 100, 200]
    all_err_means = []
    all_err_stds = []
    
    for k in ks
        error_means = []
        error_stds = []
        for n in ns
            errors = Vector{Int}([])
            for i=1:num_datasets
                dataset = draw_uniform_bimodal(R/2-k, R/2+k, n)

                xs = round_array(dataset, R)
                qs = scores(xs)
                true_median = xs[argmax(qs)]
                
                for run=1:num_runs
                    run_median = xs[RNM(qs)]
                    error = abs(score(true_median, xs) - score(run_median, xs))
                    push!(errors, error)
                end
            end  
            push!(error_means, mean(errors))
            push!(error_stds, std(errors))
        end
        push!(all_err_means, error_means)
        push!(all_err_stds, error_stds)
    end    
    plot_avgs_bimodal(ns, all_err_means, "Avg score error", "Avg score error for Bimodal runs")
    plot_avgs_bimodal(ns, all_err_stds, "Std Dev of score error", "Std Dev of score error for Bimodal runs")    
end

gaussian_exp(0.1, ns, num_datasets, num_runs)
poisson_exp(0.1, ns, num_datasets, num_runs)
bimodal_exp(0.1, ns, num_datasets, num_runs)

"/Users/bhushansuwal/Desktop/notes/privacy/hw1/Avg within-run score error for Gaussian runs.png"